# Sistem prediksi hujan pada besok harinya
Australia merupakan negara yang bisa dibilang iklimnya unik. Ada sedikit tropis di bagian utara, gersang di bagian tengah, dan dingin seperti benua eropa di bagian selatan. Hujan tentunya tidak terelakkan bagi penghuni negara tersebut. Pasti ada hal yang perlu diantisipasi. Oleh karena itu, saya akan membuat suatu classifier hujan yang dapat memprediksi apakah besok hujan atau tidak dengan mengandalkan algoritme KNN (K-Nearest Neighbours).

In [33]:
#Import semua library yang dibutuhkan 
import pandas as pd # Untuk memanipulasi file csv
import numpy as np # Untuk melakukan proses aritmetika pada matriks
import matplotlib.pyplot as plt # Untuk visualisasi pada dataset

# Membelah dataset menjadi train dan test, yang kedua untuk mencari hyperparameter yang baik
from sklearn.model_selection import train_test_split, GridSearchCV
# Model KNN nya sendiri
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
# Melakukan proses standardisasi pada suatu dataset 
from sklearn.preprocessing import StandardScaler
# Memberi label untuk tipe data kategorik
from sklearn.preprocessing import LabelEncoder
# Membantu dalam hal mengisi missing value
from sklearn.preprocessing import Imputer
# Mengukur akurasi model terhadap data yang diketahui
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

In [2]:
# Import dataset ke Python
df = pd.read_csv('../input/weatherAUS.csv')
df.head() # Melihat beberapa baris pertama dari dataset

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


In [3]:
df.shape # Mengambil nilai baris dan kolom dari dataset

(142193, 24)

Dalam kasus ini, pengguna ingin mendapatkan prediksi apakah besok hujan atau tidak tanpa memperdulikan lokasi dan waktunya kapan. Oleh karena itu, kolom __Date__ dan __Location__ dibuang menggunakan df.drop().

In [4]:
df = df.drop(['Date','Location'], axis = 1)

In [5]:
df.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 22 columns):
MinTemp          141556 non-null float64
MaxTemp          141871 non-null float64
Rainfall         140787 non-null float64
Evaporation      81350 non-null float64
Sunshine         74377 non-null float64
WindGustDir      132863 non-null object
WindGustSpeed    132923 non-null float64
WindDir9am       132180 non-null object
WindDir3pm       138415 non-null object
WindSpeed9am     140845 non-null float64
WindSpeed3pm     139563 non-null float64
Humidity9am      140419 non-null float64
Humidity3pm      138583 non-null float64
Pressure9am      128179 non-null float64
Pressure3pm      128212 non-null float64
Cloud9am         88536 non-null float64
Cloud3pm         85099 non-null float64
Temp9am          141289 non-null float64
Temp3pm          139467 non-null float64
RainToday        140787 non-null object
RISK_MM          142193 non-null float64
RainTomorrow     142193 non-null obj

Setelah kolom tersebut dibuang, langkah selanjutnya adalah mencaritau berapa missing value dari setiap kolom. Oleh karena itu dilakukan perintah berikut.

In [7]:
for col in df.columns:
    print("Kolom {}: {} ({}%)".format(col, df[col].isnull().sum(), int(df[col].isnull().sum()/df.shape[0]*100) ))

Kolom MinTemp: 637 (0%)
Kolom MaxTemp: 322 (0%)
Kolom Rainfall: 1406 (0%)
Kolom Evaporation: 60843 (42%)
Kolom Sunshine: 67816 (47%)
Kolom WindGustDir: 9330 (6%)
Kolom WindGustSpeed: 9270 (6%)
Kolom WindDir9am: 10013 (7%)
Kolom WindDir3pm: 3778 (2%)
Kolom WindSpeed9am: 1348 (0%)
Kolom WindSpeed3pm: 2630 (1%)
Kolom Humidity9am: 1774 (1%)
Kolom Humidity3pm: 3610 (2%)
Kolom Pressure9am: 14014 (9%)
Kolom Pressure3pm: 13981 (9%)
Kolom Cloud9am: 53657 (37%)
Kolom Cloud3pm: 57094 (40%)
Kolom Temp9am: 904 (0%)
Kolom Temp3pm: 2726 (1%)
Kolom RainToday: 1406 (0%)
Kolom RISK_MM: 0 (0%)
Kolom RainTomorrow: 0 (0%)


Dapat dilihat, hampir semua kolom terdapat missing value. Ada kolom yang missing value nya mencapai 40% dari data. Untuk data tersebut, kemungkinan besar akan dibuang. Kolom tersebut, yakni Evaporation, Sunshine, Cloud9am, dan Cloud3pm.

In [8]:
df = df.drop(['Evaporation','Sunshine', 'Cloud9am', 'Cloud3pm'], axis = 1)

In [9]:
df.head()

,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,13.4,22.9,0.6,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,16.9,21.8,No,0.0,No
1,7.4,25.1,0.0,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,17.2,24.3,No,0.0,No
2,12.9,25.7,0.0,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,21.0,23.2,No,0.0,No
3,9.2,28.0,0.0,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,18.1,26.5,No,1.0,No
4,17.5,32.3,1.0,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,17.8,29.7,No,0.2,No


Setelah membuang kolom tersebut, maka dilihat kolom yang tidak saling bebas atau memiliki korelasi.

In [10]:
# pd.plotting.scatter_matrix(df, hist_kwds = {'bins':20}, marker = 'o')
# plt.show()

# Tidak dijalankan karena prosesnya kelamaan dan ribet

Dengan visualisasi yang tidak jelas maksudnya apa, dapat disimpulkan terdapat korelasi yang kuat antara temp3am dan temp 3pm. Begitu juga dengan pressure. Sehingga salah satu kolom dapat dibuang.

In [11]:
df = df.drop(['Pressure3pm','Temp3pm'], axis = 1)

In [12]:
print(df.shape)
print(df.columns)

(142193, 16)
Index(['MinTemp', 'MaxTemp', 'Rainfall', 'WindGustDir', 'WindGustSpeed',
       'WindDir9am', 'WindDir3pm', 'WindSpeed9am', 'WindSpeed3pm',
       'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Temp9am', 'RainToday',
       'RISK_MM', 'RainTomorrow'],
      dtype='object')


Karena data yang digunakan berupa data kuantitatif atau data numerik, maka data kategorik seperti WindGustDir, WindDir9am, WindDir3pm, dan RainToday akan dihapus.

In [13]:
df = df.drop(['WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday'], axis = 1)

In [14]:
df.head()

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Temp9am,RISK_MM,RainTomorrow
0,13.4,22.9,0.6,44.0,20.0,24.0,71.0,22.0,1007.7,16.9,0.0,No
1,7.4,25.1,0.0,44.0,4.0,22.0,44.0,25.0,1010.6,17.2,0.0,No
2,12.9,25.7,0.0,46.0,19.0,26.0,38.0,30.0,1007.6,21.0,0.0,No
3,9.2,28.0,0.0,24.0,11.0,9.0,45.0,16.0,1017.6,18.1,1.0,No
4,17.5,32.3,1.0,41.0,7.0,20.0,82.0,33.0,1010.8,17.8,0.2,No


In [15]:
df.shape

(142193, 12)

Setelah dataset sudah diproses, maka langkah berikutnya adalah memisahkan variabel biasa dengan variabel label atau targetnya seperti berikut

In [18]:
X = df.iloc[:, 0:11]
X.head()

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Temp9am,RISK_MM
0,13.4,22.9,0.6,44.0,20.0,24.0,71.0,22.0,1007.7,16.9,0.0
1,7.4,25.1,0.0,44.0,4.0,22.0,44.0,25.0,1010.6,17.2,0.0
2,12.9,25.7,0.0,46.0,19.0,26.0,38.0,30.0,1007.6,21.0,0.0
3,9.2,28.0,0.0,24.0,11.0,9.0,45.0,16.0,1017.6,18.1,1.0
4,17.5,32.3,1.0,41.0,7.0,20.0,82.0,33.0,1010.8,17.8,0.2


Catatan: Variabel Target (RainTomorrow) tersebut masih dalam bentuk teks, oleh karena itu diperlukan pelabelan menggunakan _LabelEncoder()_

In [21]:
le = LabelEncoder()
y = le.fit_transform(df.RainTomorrow)
print(y)

[0 0 0 ... 0 0 0]


Setelah X dan y dipisah, langkah berikutnya adalah memisahkan dataset menjadi bagian train dan test. Bagian train akan digunakan dalam pembuatan model, sedangkan untuk test akan digunakan untuk melihat seberapa bagus model yang telah dibuat.

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

Setelah dipisah ke dalam train dan test, maka hal yang perlu dilakukan adalah Membuat model. Untuk membuat model, disini akan digunakan objek Pipeline() dan objek GridSearchCV() dengan model yang digunakan adalah KNN dimana kalau di Python menggunakan objek KNeighborsClassifier().

In [25]:
steps = [
    ('imp', Imputer(missing_values = 'NaN', strategy = 'median', axis = 0)),
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors = 5))
]
# parameters = {'knn__n_neighbors':np.arange(1,50)}
pipeline = Pipeline(steps)
# cv = GridSearchCV(pipeline, param_grid = parameters)
pipeline.fit(X_train, y_train)


/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


Pipeline(memory=None,
     steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'))])

Berikut Hasil Akurasi jika dataset telah dilakukan Standardisasi menggunakan StandardScaler()

In [36]:
y_pred = pipeline.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred, labels=[0, 1]))

0.908541087942614
[[21644   454]
 [ 2147  4194]]


Setelah mengetahui akurasi 90.8%, apakah bisa dikatakan model tersebut bagus? Belum tentu Maria, maka digunakan lagi satu metric yang akan menentukan, yakni __precision__ dan __recall__. 
- Precision adalah persentase True Negative dibagi dengan jumlah dari True Negative dan False Negative atau sebaliknya
- Recall adalah persentase True Negative dibagi dengan jumlah dari True Negative dan False Positive

In [37]:
recall = 21644 / (21644 + 2147)
precision = 21644 / (21644 + 454)
print('Recall {:.2f}'.format(recall * 100))
print('Precision {:.2f}'.format(precision * 100))

Recall 90.98
Precision 97.95


Wow, hasil yang cukup mengejutkan! Model ini bisa memprediksi besok tidak hujan dengan presisi yang cukup tinggi, yakni 97.95%

Berikut Hasil Akurasi tanpa StandardScaler()

In [28]:
steps = [
    ('imp', Imputer(missing_values = 'NaN', strategy = 'median', axis = 0)),
    ('knn', KNeighborsClassifier(n_neighbors= 3))
]
pipeline2 = Pipeline(steps)
pipeline2.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


Pipeline(memory=None,
     steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform'))])

In [29]:
y_pred2 = pipeline2.predict(X_test)
accuracy_score(y_test, y_pred2)

0.8919441611871022